<a href="https://colab.research.google.com/github/Lukey-B/Lukes_Projects/blob/main/Working_With_File_Formats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
<img src="https://drive.google.com/uc?export=view&id=15Bu95rywbHkN5od0ocRWAbi1EBiQb-Z2" width="400"/>
</div>

In this tutorial we will take a quick look at how to handle files in python. We'll focus mainly on csv and Json files and how we can use the pandas package to handle these files.

### 1) **Working with text files**

Python has built-in functions to read and open files. The file object can provide you with a fair bit of information about the file you're opening.

In [ ]:
with open('demo.txt', 'w+') as f:
    print ("What was the name of the file? ", f.name)
    print ("Is the file closed?", f.closed)
    f.write("Hello World")

    f.close()

Let's open the file we created above and see what we wrote to it.

In [ ]:
with open('demo.txt') as f2:
    contents = f2.read()
    print (contents)

    f2.close()

Next let's take a look at CSV files

### 2) **Comma-separated values (and other text delimiters)**

There are multiple ways to handle CSVs using Python. First let's download a CSV file to work with:



In [ ]:
!wget https://my-traffic-web.s3.amazonaws.com/Traffic_Counts_at_Permanent_stations.csv

You should now see a file called 'Traffic_Counts_at_Permanent_stations.csv' in your files tab (to the left).

We can use Python built-ins to open and handle this CSV ourselves. Let's write some code to return the first five lines of the provided file below:

In [ ]:
with open('Traffic_Counts_at_Permanent_stations.csv') as f3:

    # Read the first line
    header = f3.readline()
    print(header)

    # Read the second
    line1 = f3.readline()
    print(line1)

    # Use split to get a better format
    line2 = f3.readline()
    print(line2.split(','))

    # Close the file
    f3.close()

We can use `pandas.read_csv()` to automate this process and give us more functionality when reading a CSV into a dataframe

In [ ]:
import pandas as pd

df = pd.read_csv("Traffic_Counts_at_Permanent_stations.csv")

df.head()



For large files, it's often desirable to break up the file a bit at a time for processing. This can help you use less memory.

In [ ]:
reader = pd.read_csv("Traffic_Counts_at_Permanent_stations.csv", chunksize = 500000)
reader

By specifying the chuncksize pandas will not read the whole csv into memory, but rather will create a TextFileReader to access parts of the dataframe.

In [ ]:
#gets the 10th chunk of the dataframe
reader.get_chunk(10)

In [ ]:
#let's take a look at headers of these dataframes
for chunk in reader:
    print (chunk.head(2))

<font color='red'>**Things to try**</font>

* One of the ways you can use this is to break up your file into chunks, then for each chunk, filter only for rows/columns that you're interested in. Then you can use `concat()` to amalgamate your data together.


In [ ]:
# Try filtering out data so we only get data in the NE
reader = pd.read_csv("Traffic_Counts_at_Permanent_stations.csv", chunksize = 500000)
neData = pd.DataFrame()

df_list = []
for chunk in reader:
  mask = chunk['ADDRESS'].str.contains("NE")
  filtered_chunk = chunk.loc[mask]
  df_list.append(filtered_chunk)



In [ ]:
pd.concat(df_list)

* (optional) Is it possible to open the file using Python's `open()` method and then read it into a dataframe yourself? What are the benefits to doing this? What would be the drawbacks?

In [ ]:
# of course it is possible - let's try it
%%timeit
df = pd.DataFrame()
lines = []
with open('Traffic_Counts_at_Permanent_stations.csv') as f:
    for line in f:
      line_split = line.split(",")
      lines.append(line_split)

pd.DataFrame(lines)

In [ ]:
%%timeit

reader = pd.read_csv("Traffic_Counts_at_Permanent_stations.csv", chunksize = 500000)
neData = pd.DataFrame()

df_list = []
for chunk in reader:
  df_list.append(chunk)

pd.concat(df_list)

The benefits are the native open function is a bit faster. The drawbacks are that it might be a couple extra steps

### 3) **JSON Documents**

There are both Python built-in and pandas methods to handle document formats such as JSON. Note, however, not all JSON data works particularly well with pandas, because they are documents rather than tables. This is especially the case when you have JSON files that are highly nested.

Let's create a simple JSON file



In [ ]:
import json

#made up data
data = {
    "name": "John Doe",
    "age": 30,
    "is_employee": True,
    "address": {
        "street": "123 Main St",
        "city": "Anytown"
    },
    "phone_numbers": {"home":"123-456-7890", "cell":"456-789-0123"}
}

#write the json file
with open('data.json', 'w') as f:
    json.dump(data, f, indent=4)

Notice however in this json file we have a few values nested within both address and phone_numbers. Let's try and read

In [ ]:
with open('data.json', 'r') as file:
  print(json.load(file))

{'name': 'John Doe', 'age': 30, 'is_employee': True, 'address': {'street': '123 Main St', 'city': 'Anytown'}, 'phone_numbers': {'home': '123-456-7890', 'cell': '456-789-0123'}}


Let's now try to import this data into a pandas dataframe.

In [ ]:
import pandas as pd
df = pd.read_json("data.json")
df

,name,age,is_employee,address,phone_numbers
street,John Doe,30,True,123 Main St,NaN
city,John Doe,30,True,Anytown,NaN
home,John Doe,30,True,NaN,123-456-7890
cell,John Doe,30,True,NaN,456-789-0123


We can see that all the data is there, though we might want to rearrange the table to reduce redundancy.

<font color='red'>**Things to try**</font>

* Try rearranging the table to reduce redundancy.

* Try adding more information to the json data and see if you can import it into a dataframe in a useful format.

* Try using [the JSON generator](https://www.json-generator.com/) to build your own JSON file and practice using these methods.

In [ ]:
# rearranging table
df.set_index("name",inplace = True)
df.reset_index(inplace = True)

In [ ]:
# applying lambda function to move null values to bottom of the columns
df = df.apply(lambda x: pd.Series(x.dropna().values))
df.drop(index = [2,3],inplace = True)
df

,name,age,is_employee,address,phone_numbers
0,John Doe,30,True,123 Main St,123-456-7890
1,John Doe,30,True,Anytown,456-789-0123


In [ ]:
# manually creating modifying table structure and values
df["address"] = df["address"].iloc[0] + "," + df["address"].iloc[1]
df["home_phone"] = df["phone_numbers"].iloc[0]
df["cell_phone"] = df["phone_numbers"].iloc[1]
df.drop(columns = "phone_numbers",inplace = True)
df.drop(index= 1)

,name,age,is_employee,address,home_phone,cell_phone
0,John Doe,30,True,"123 Main St,Anytown",123-456-7890,456-789-0123


In [ ]:
# using json_normalize to flatten data structure creating seperate for each of the keys in the nested json object
pd.json_normalize(data)

,name,age,is_employee,address.street,address.city,phone_numbers.home,phone_numbers.cell
0,John Doe,30,True,123 Main St,Anytown,123-456-7890,456-789-0123


In [ ]:
import json

# adding more data
data = {
    "name": ["John Doe","Luke B","John Wick"],
    "age": [30,28,40],
    "is_employee": [True,False,False],
    "address": [{
        "street": "123 Main St",
        "city": "Anytown"
    },{
        "street": "123 Main St",
        "city": "Anytown"
    },{
        "street": "123 Main St",
        "city": "Anytown"
    }],
    "phone_numbers": [{"home":"123-456-7890", "cell":"456-789-0123"},
     {"home":"123-456-7890", "cell":"456-789-0123"},
      {"home":"123-456-7890", "cell":"456-789-0123"}]
}

#write the json file
with open('data.json', 'w') as f:
    json.dump(data, f, indent=4)

The introduction of lists into the data structure is useful as pandas will interpret each list as it's own column, with each item in the list corresponding to its own row

In [ ]:
pd.read_json("data.json")

,name,age,is_employee,address,phone_numbers
0,John Doe,30,True,"{'street': '123 Main St', 'city': 'Anytown'}","{'home': '123-456-7890', 'cell': '456-789-0123'}"
1,Luke B,28,False,"{'street': '123 Main St', 'city': 'Anytown'}","{'home': '123-456-7890', 'cell': '456-789-0123'}"
2,John Wick,40,False,"{'street': '123 Main St', 'city': 'Anytown'}","{'home': '123-456-7890', 'cell': '456-789-0123'}"


In [ ]:
# randomly generated json data

data = [
  {
    "_id": "65e0ca922eef1a103094bc17",
    "index": 0,
    "guid": "a66d05d0-f4d9-49e5-aea2-0dc62880bdff",
    "isActive": False,
    "balance": "$2,824.29",
    "picture": "http://placehold.it/32x32",
    "age": 40,
    "eyeColor": "blue",
    "name": "Odessa Walls",
    "gender": "female",
    "company": "DENTREX",
    "email": "odessawalls@dentrex.com",
    "phone": "+1 (918) 435-3153",
    "address": "477 Independence Avenue, Albany, Michigan, 3042",
    "about": "Culpa voluptate incididunt elit proident anim proident voluptate aute exercitation laboris sint nisi anim. Esse nulla in veniam in proident labore exercitation ut aliquip amet. Aliquip magna id enim consequat aliqua adipisicing ex. Quis enim voluptate sunt eu nisi. Labore excepteur do duis exercitation consectetur sunt nisi in. Aliquip ex sit velit cupidatat occaecat sint ut amet eiusmod in magna.\r\n",
    "registered": "2014-11-20T09:13:52 +07:00",
    "latitude": 11.97649,
    "longitude": 82.628315,
    "tags": [
      "aliquip",
      "consectetur",
      "veniam",
      "quis",
      "velit",
      "occaecat",
      "magna"
    ],
    "friends": [
      {
        "id": 0,
        "name": "Whitehead Scott"
      },
      {
        "id": 1,
        "name": "Ericka Ramsey"
      },
      {
        "id": 2,
        "name": "Ava Mccullough"
      }
    ],
    "greeting": "Hello, Odessa Walls! You have 7 unread messages.",
    "favoriteFruit": "banana"
  },
  {
    "_id": "65e0ca92658db3b7096b38fe",
    "index": 1,
    "guid": "ab562d41-9058-493b-9a6d-270cde0f54f6",
    "isActive": True,
    "balance": "$1,478.83",
    "picture": "http://placehold.it/32x32",
    "age": 24,
    "eyeColor": "blue",
    "name": "Krista Perry",
    "gender": "female",
    "company": "EGYPTO",
    "email": "kristaperry@egypto.com",
    "phone": "+1 (809) 581-3725",
    "address": "906 Gotham Avenue, Abiquiu, Alaska, 4322",
    "about": "Dolor ullamco amet non elit veniam esse ipsum laboris. Et occaecat aliquip veniam exercitation veniam magna qui voluptate est ut laborum eiusmod. Anim ullamco do ipsum ex eiusmod reprehenderit nulla enim nulla ea excepteur. Aliquip ea sint esse tempor. Consectetur amet incididunt id laborum minim culpa exercitation. Ut aute tempor non nulla qui non minim aliquip cupidatat sunt cupidatat fugiat ex.\r\n",
    "registered": "2018-02-10T05:10:08 +07:00",
    "latitude": 55.852849,
    "longitude": -132.003489,
    "tags": [
      "occaecat",
      "tempor",
      "est",
      "est",
      "commodo",
      "aliquip",
      "ut"
    ],
    "friends": [
      {
        "id": 0,
        "name": "Letitia Mcdowell"
      },
      {
        "id": 1,
        "name": "Armstrong Blair"
      },
      {
        "id": 2,
        "name": "Padilla Barrett"
      }
    ],
    "greeting": "Hello, Krista Perry! You have 4 unread messages.",
    "favoriteFruit": "apple"
  },
  {
    "_id": "65e0ca92ddfcde46781b6181",
    "index": 2,
    "guid": "c2e2aaa1-dd95-4396-9e91-54fc6093cfc1",
    "isActive": True,
    "balance": "$3,450.00",
    "picture": "http://placehold.it/32x32",
    "age": 21,
    "eyeColor": "green",
    "name": "Dickson Barrera",
    "gender": "male",
    "company": "SUNCLIPSE",
    "email": "dicksonbarrera@sunclipse.com",
    "phone": "+1 (816) 496-3047",
    "address": "424 Albemarle Terrace, Goodville, Oregon, 6694",
    "about": "Nostrud occaecat sunt eu mollit voluptate nisi eu ullamco. Minim aute pariatur reprehenderit culpa deserunt consectetur enim nostrud aute Lorem do irure tempor esse. Irure quis ullamco do aliquip ea ea. Do minim et quis et duis.\r\n",
    "registered": "2016-08-16T08:42:30 +06:00",
    "latitude": -48.747361,
    "longitude": -85.79685,
    "tags": [
      "consectetur",
      "et",
      "cillum",
      "ullamco",
      "non",
      "dolore",
      "adipisicing"
    ],
    "friends": [
      {
        "id": 0,
        "name": "Colette Mcmillan"
      },
      {
        "id": 1,
        "name": "Clark Pickett"
      },
      {
        "id": 2,
        "name": "Summers Bruce"
      }
    ],
    "greeting": "Hello, Dickson Barrera! You have 5 unread messages.",
    "favoriteFruit": "banana"
  },
  {
    "_id": "65e0ca928023047ac0042619",
    "index": 3,
    "guid": "6ff62e74-ae92-4328-a9de-d3a35a0c0556",
    "isActive": True,
    "balance": "$1,599.69",
    "picture": "http://placehold.it/32x32",
    "age": 28,
    "eyeColor": "brown",
    "name": "Kenya Nolan",
    "gender": "female",
    "company": "EXTRO",
    "email": "kenyanolan@extro.com",
    "phone": "+1 (972) 527-3076",
    "address": "943 Polhemus Place, Sedley, Ohio, 2737",
    "about": "Enim sint deserunt occaecat adipisicing veniam ad labore ex amet Lorem cillum anim sit. Deserunt officia quis eu quis non fugiat nisi commodo minim minim velit laborum adipisicing reprehenderit. Ea ullamco nulla eu aute cillum do officia aliquip nisi commodo irure fugiat nisi est. Esse proident ullamco velit laboris est quis laborum do ad. Incididunt id enim aliquip ex cupidatat consequat dolor et ut eiusmod magna.\r\n",
    "registered": "2016-08-13T07:34:40 +06:00",
    "latitude": 73.549308,
    "longitude": -72.335005,
    "tags": [
      "labore",
      "ut",
      "velit",
      "enim",
      "tempor",
      "enim",
      "voluptate"
    ],
    "friends": [
      {
        "id": 0,
        "name": "Kathrine Miranda"
      },
      {
        "id": 1,
        "name": "Floyd Dickerson"
      },
      {
        "id": 2,
        "name": "Blevins Sutton"
      }
    ],
    "greeting": "Hello, Kenya Nolan! You have 4 unread messages.",
    "favoriteFruit": "strawberry"
  },
  {
    "_id": "65e0ca925a4cbe94a3797c12",
    "index": 4,
    "guid": "0ba1b1b6-315d-4a5d-935b-dabe915598f4",
    "isActive": False,
    "balance": "$1,278.53",
    "picture": "http://placehold.it/32x32",
    "age": 40,
    "eyeColor": "green",
    "name": "Shaffer Roth",
    "gender": "male",
    "company": "PROSURE",
    "email": "shafferroth@prosure.com",
    "phone": "+1 (970) 430-2007",
    "address": "659 Coventry Road, Hinsdale, Montana, 2069",
    "about": "Minim excepteur magna irure aliqua nostrud. Nulla ea esse irure irure nostrud aliqua voluptate fugiat cupidatat est cupidatat non ea. Aute aliqua voluptate laboris tempor eu consequat adipisicing anim laboris excepteur eiusmod tempor nulla.\r\n",
    "registered": "2016-06-08T06:45:22 +06:00",
    "latitude": 17.120083,
    "longitude": -4.429316,
    "tags": [
      "cillum",
      "anim",
      "nisi",
      "commodo",
      "amet",
      "nostrud",
      "incididunt"
    ],
    "friends": [
      {
        "id": 0,
        "name": "Rosalinda Rhodes"
      },
      {
        "id": 1,
        "name": "Blanchard Morris"
      },
      {
        "id": 2,
        "name": "Frye Woods"
      }
    ],
    "greeting": "Hello, Shaffer Roth! You have 5 unread messages.",
    "favoriteFruit": "strawberry"
  },
  {
    "_id": "65e0ca92c029e976c3286e74",
    "index": 5,
    "guid": "327048a4-3b70-4033-b7c6-cbd88086fdb4",
    "isActive": True,
    "balance": "$1,992.96",
    "picture": "http://placehold.it/32x32",
    "age": 22,
    "eyeColor": "blue",
    "name": "House Sosa",
    "gender": "male",
    "company": "NETERIA",
    "email": "housesosa@neteria.com",
    "phone": "+1 (902) 592-2953",
    "address": "252 Ocean Court, Naomi, Northern Mariana Islands, 7591",
    "about": "Pariatur sint incididunt excepteur commodo incididunt cupidatat quis ut. Laboris in labore consequat nostrud ea aliquip anim dolore eiusmod esse aute. Mollit voluptate Lorem qui occaecat fugiat labore qui pariatur aliquip sint. In quis sint do aliquip est non eiusmod sint Lorem. Ipsum nostrud voluptate enim esse dolor nisi aute nostrud excepteur reprehenderit adipisicing.\r\n",
    "registered": "2019-12-22T09:13:52 +07:00",
    "latitude": 7.607462,
    "longitude": -31.423068,
    "tags": [
      "consectetur",
      "aliqua",
      "cillum",
      "sint",
      "nulla",
      "velit",
      "est"
    ],
    "friends": [
      {
        "id": 0,
        "name": "Alyssa Shepard"
      },
      {
        "id": 1,
        "name": "Rivera Franks"
      },
      {
        "id": 2,
        "name": "Jewel Boyle"
      }
    ],
    "greeting": "Hello, House Sosa! You have 7 unread messages.",
    "favoriteFruit": "banana"
  }
]

In [ ]:
with open('data.json', 'w') as f:
    json.dump(data, f, indent=4)

In [ ]:
rand_df = pd.read_json("data.json")
rand_df.head()

,_id,index,guid,isActive,balance,picture,age,eyeColor,name,gender,...,phone,address,about,registered,latitude,longitude,tags,friends,greeting,favoriteFruit
0,65e0ca922eef1a103094bc17,0,a66d05d0-f4d9-49e5-aea2-0dc62880bdff,False,"$2,824.29",http://placehold.it/32x32,40,blue,Odessa Walls,female,...,+1 (918) 435-3153,"477 Independence Avenue, Albany, Michigan, 3042",Culpa voluptate incididunt elit proident anim ...,2014-11-20T09:13:52 +07:00,11.976490,82.628315,"[aliquip, consectetur, veniam, quis, velit, oc...","[{'id': 0, 'name': 'Whitehead Scott'}, {'id': ...","Hello, Odessa Walls! You have 7 unread messages.",banana
1,65e0ca92658db3b7096b38fe,1,ab562d41-9058-493b-9a6d-270cde0f54f6,True,"$1,478.83",http://placehold.it/32x32,24,blue,Krista Perry,female,...,+1 (809) 581-3725,"906 Gotham Avenue, Abiquiu, Alaska, 4322",Dolor ullamco amet non elit veniam esse ipsum ...,2018-02-10T05:10:08 +07:00,55.852849,-132.003489,"[occaecat, tempor, est, est, commodo, aliquip,...","[{'id': 0, 'name': 'Letitia Mcdowell'}, {'id':...","Hello, Krista Perry! You have 4 unread messages.",apple
2,65e0ca92ddfcde46781b6181,2,c2e2aaa1-dd95-4396-9e91-54fc6093cfc1,True,"$3,450.00",http://placehold.it/32x32,21,green,Dickson Barrera,male,...,+1 (816) 496-3047,"424 Albemarle Terrace, Goodville, Oregon, 6694",Nostrud occaecat sunt eu mollit voluptate nisi...,2016-08-16T08:42:30 +06:00,-48.747361,-85.796850,"[consectetur, et, cillum, ullamco, non, dolore...","[{'id': 0, 'name': 'Colette Mcmillan'}, {'id':...","Hello, Dickson Barrera! You have 5 unread mess...",banana
3,65e0ca928023047ac0042619,3,6ff62e74-ae92-4328-a9de-d3a35a0c0556,True,"$1,599.69",http://placehold.it/32x32,28,brown,Kenya Nolan,female,...,+1 (972) 527-3076,"943 Polhemus Place, Sedley, Ohio, 2737",Enim sint deserunt occaecat adipisicing veniam...,2016-08-13T07:34:40 +06:00,73.549308,-72.335005,"[labore, ut, velit, enim, tempor, enim, volupt...","[{'id': 0, 'name': 'Kathrine Miranda'}, {'id':...","Hello, Kenya Nolan! You have 4 unread messages.",strawberry
4,65e0ca925a4cbe94a3797c12,4,0ba1b1b6-315d-4a5d-935b-dabe915598f4,False,"$1,278.53",http://placehold.it/32x32,40,green,Shaffer Roth,male,...,+1 (970) 430-2007,"659 Coventry Road, Hinsdale, Montana, 2069",Minim excepteur magna irure aliqua nostrud. Nu...,2016-06-08T06:45:22 +06:00,17.120083,-4.429316,"[cillum, anim, nisi, commodo, amet, nostrud, i...","[{'id': 0, 'name': 'Rosalinda Rhodes'}, {'id':...","Hello, Shaffer Roth! You have 5 unread messages.",strawberry


The Data read in quite nicely

In [ ]:
rand_df.describe()
rand_df.shape
rand_df.dtypes

RangeIndex(start=0, stop=6, step=1)